In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    FFF_Train_Decile4 = pd.read_pickle("FFF_Train_Decile4.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile4.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile4, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 4')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 3s 1ms/step - loss: 0.4504 - acc: 0.7967 - val_loss: 0.4065 - val_acc: 0.8207
Epoch 39/100
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4517 - acc: 0.7997 - val_loss: 0.4121 - val_acc: 0.8181
Epoch 40/100
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4467 - acc: 0.8043 - val_loss: 0.4056 - val_acc: 0.8168
Epoch 41/100
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4463 - acc: 0.8039 - val_loss: 0.4031 - val_acc: 0.8181
Epoch 42/100
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4503 - acc: 0.7974 - val_loss: 0.4049 - val_acc: 0.8246
Epoch 43/100
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4463 - acc: 0.8062 - val_loss: 0.4011 - val_acc: 0.8220
Epoch 44/100
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4434 - acc: 0.8026 - val_loss: 0.4054 - val_acc: 0.8181
Epoch 45/100
3055/3055 [==============================] - 3s 

Epoch 98/100
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4327 - acc: 0.8023 - val_loss: 0.3888 - val_acc: 0.8325
Epoch 99/100
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4334 - acc: 0.8065 - val_loss: 0.3889 - val_acc: 0.8338
Epoch 100/100
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4359 - acc: 0.8059 - val_loss: 0.3892 - val_acc: 0.8325
Test accuracy: 0.8324607320480946
Train on 3055 samples, validate on 764 samples
Epoch 1/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.6127 - acc: 0.6501 - val_loss: 0.5164 - val_acc: 0.7264
Epoch 2/150
3055/3055 [==============================] - 3s 1ms/step - loss: 0.5101 - acc: 0.7604 - val_loss: 0.4588 - val_acc: 0.7958
Epoch 3/150
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4924 - acc: 0.7859 - val_loss: 0.4285 - val_acc: 0.8194
Epoch 4/150
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4694 - acc: 0.7997 - val_l

3055/3055 [==============================] - 3s 1ms/step - loss: 0.4113 - acc: 0.8187 - val_loss: 0.3733 - val_acc: 0.8325
Epoch 57/150
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4137 - acc: 0.8216 - val_loss: 0.3734 - val_acc: 0.8325
Epoch 58/150
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4115 - acc: 0.8232 - val_loss: 0.3733 - val_acc: 0.8325
Epoch 59/150
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4114 - acc: 0.8196 - val_loss: 0.3732 - val_acc: 0.8338
Epoch 60/150
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4192 - acc: 0.8206 - val_loss: 0.3733 - val_acc: 0.8338
Epoch 61/150
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4141 - acc: 0.8219 - val_loss: 0.3735 - val_acc: 0.8338
Epoch 62/150
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4120 - acc: 0.8183 - val_loss: 0.3733 - val_acc: 0.8325
Epoch 63/150
3055/3055 [==============================] - 3s 

3055/3055 [==============================] - 3s 1ms/step - loss: 0.4369 - acc: 0.8121 - val_loss: 0.3840 - val_acc: 0.8259
Epoch 26/150
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4296 - acc: 0.8147 - val_loss: 0.4009 - val_acc: 0.8233
Epoch 27/150
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4321 - acc: 0.8180 - val_loss: 0.3808 - val_acc: 0.8220
Epoch 28/150
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4265 - acc: 0.8121 - val_loss: 0.3886 - val_acc: 0.8272
Epoch 29/150
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4279 - acc: 0.8121 - val_loss: 0.3817 - val_acc: 0.8233
Epoch 30/150
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4273 - acc: 0.8154 - val_loss: 0.3791 - val_acc: 0.8246
Epoch 31/150
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4257 - acc: 0.8160 - val_loss: 0.3872 - val_acc: 0.8259
Epoch 32/150
3055/3055 [==============================] - 3s 

...

3055/3055 [==============================] - 3s 1ms/step - loss: 0.4047 - acc: 0.8196 - val_loss: 0.3640 - val_acc: 0.8403
Epoch 87/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4063 - acc: 0.8200 - val_loss: 0.3638 - val_acc: 0.8403
Epoch 88/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4042 - acc: 0.8295 - val_loss: 0.3637 - val_acc: 0.8403
Epoch 89/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4037 - acc: 0.8272 - val_loss: 0.3637 - val_acc: 0.8416
Epoch 90/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4060 - acc: 0.8200 - val_loss: 0.3636 - val_acc: 0.8390
Epoch 91/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4037 - acc: 0.8213 - val_loss: 0.3635 - val_acc: 0.8390
Epoch 92/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4046 - acc: 0.8249 - val_loss: 0.3637 - val_acc: 0.8416
Epoch 93/200
3055/3055 [==============================] - 3s 

Epoch 146/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4081 - acc: 0.8219 - val_loss: 0.3634 - val_acc: 0.8390
Epoch 147/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.3994 - acc: 0.8216 - val_loss: 0.3633 - val_acc: 0.8403
Epoch 148/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4072 - acc: 0.8193 - val_loss: 0.3634 - val_acc: 0.8416
Epoch 149/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4024 - acc: 0.8203 - val_loss: 0.3632 - val_acc: 0.8416
Epoch 150/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4020 - acc: 0.8196 - val_loss: 0.3631 - val_acc: 0.8429
Epoch 151/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4055 - acc: 0.8268 - val_loss: 0.3631 - val_acc: 0.8429
Epoch 152/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4064 - acc: 0.8206 - val_loss: 0.3630 - val_acc: 0.8403
Epoch 153/200
3055/3055 [================

Epoch 65/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4682 - acc: 0.7993 - val_loss: 0.4234 - val_acc: 0.8076
Epoch 66/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4685 - acc: 0.8000 - val_loss: 0.4238 - val_acc: 0.8076
Epoch 67/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4624 - acc: 0.7993 - val_loss: 0.4233 - val_acc: 0.8063
Epoch 68/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4619 - acc: 0.7967 - val_loss: 0.4231 - val_acc: 0.8063
Epoch 69/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4630 - acc: 0.8026 - val_loss: 0.4229 - val_acc: 0.8089
Epoch 70/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4563 - acc: 0.7957 - val_loss: 0.4228 - val_acc: 0.8063
Epoch 71/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4687 - acc: 0.7993 - val_loss: 0.4227 - val_acc: 0.8063
Epoch 72/100
3055/3055 [========================

Epoch 25/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4179 - acc: 0.8242 - val_loss: 0.3673 - val_acc: 0.8364

Epoch 00025: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 26/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4070 - acc: 0.8209 - val_loss: 0.3640 - val_acc: 0.8364
Epoch 27/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4125 - acc: 0.8177 - val_loss: 0.3631 - val_acc: 0.8377
Epoch 28/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4070 - acc: 0.8173 - val_loss: 0.3647 - val_acc: 0.8364
Epoch 29/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4051 - acc: 0.8216 - val_loss: 0.3639 - val_acc: 0.8390
Epoch 30/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4015 - acc: 0.8193 - val_loss: 0.3620 - val_acc: 0.8390
Epoch 31/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4096 - acc: 0.8190 - v

3055/3055 [==============================] - 3s 1ms/step - loss: 0.4075 - acc: 0.8255 - val_loss: 0.3622 - val_acc: 0.8429
Epoch 144/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4006 - acc: 0.8206 - val_loss: 0.3622 - val_acc: 0.8429
Epoch 145/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4073 - acc: 0.8203 - val_loss: 0.3622 - val_acc: 0.8429
Epoch 146/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4037 - acc: 0.8200 - val_loss: 0.3621 - val_acc: 0.8429
Epoch 147/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4026 - acc: 0.8249 - val_loss: 0.3621 - val_acc: 0.8429
Epoch 148/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4018 - acc: 0.8229 - val_loss: 0.3618 - val_acc: 0.8416
Epoch 149/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4088 - acc: 0.8219 - val_loss: 0.3622 - val_acc: 0.8429
Epoch 150/200
3055/3055 [==============================

In [3]:
0.93484

0.93484

In [ ]:
#0.85